In [2]:
import numpy as np
import pandas as pd

In [4]:
df = pd.read_csv('http://bit.ly/kaggletrain')

In [8]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [6]:
df.shape

(891, 12)

In [7]:
df.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [9]:
df = df.loc[df.Embarked.notna(), ['Survived', 'Pclass', 'Sex', 'SibSp', 'Parch', 'Fare', 'Embarked']]

In [10]:
df.head()

,Survived,Pclass,Sex,SibSp,Parch,Fare,Embarked
0,0,3,male,1,0,7.2500,S
1,1,1,female,1,0,71.2833,C
2,1,3,female,0,0,7.9250,S
3,1,1,female,1,0,53.1000,S
4,0,3,male,0,0,8.0500,S


In [11]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.svm import SVR
from sklearn.feature_selection import f_regression, mutual_info_regression, SelectKBest, RFE

In [30]:
# Didn't put 'PClass' into any of the feature lists.
# The parameter `remainder='passthrough'` allows the unlisted feature to continue into final df

nbr_feats = [ 'SibSp', 'Parch', 'Fare']
cat_feats = ['Sex', 'Embarked']

# Method 1: Split numeric and categorical handling into separate pipelines
# This is only needed if each set of features needs more than one kind of transformation

# nbr_transformer = Pipeline(
#     steps=[('scaler', StandardScaler())])

# cat_transformer = Pipeline(
#     steps=[('encoder', OneHotEncoder())])

# col_transformers = ColumnTransformer(
#     transformers=[
#         ('nbr', nbr_transformer, nbr_feats),
#         ('cat', cat_transformer, cat_feats)])


# Method 2: Put transformers directly into the ColumnTransformer
# Note, you lose the column names. Will need to figure this out later

col_transformer_2 = ColumnTransformer(
    transformers=[
        ('scaler', StandardScaler(), nbr_feats),
        ('encoder', OneHotEncoder(), cat_feats)],
    remainder='passthrough')


# Feature reduction not playing nicely with other pipeline steps. Can figure this out later.

# reduce_transformer = Pipeline(
#     steps=[
#         ('svr', RFE(SVR(kernel="linear"))),# default: eliminate 50%
#         ('f_reg',SelectKBest(f_regression, k=4)),
#         ('mut_info',SelectKBest(mutual_info_regression, k=4))
#     ])

# pipe = Pipeline(
#     steps=[
#         ('transform_cols', col_transformers),
#         ('reduce_feats', reduce_transformer)])

In [17]:
X = df.drop('Survived', axis=1)
y = df.Survived

In [27]:
col_transformers.fit_transform(X)

array([[ 0.82520863,  0.43135024, -0.47432585, ...,  0.        ,
         0.        ,  1.        ],
       [-1.57221121,  0.43135024, -0.47432585, ...,  1.        ,
         0.        ,  0.        ],
       [ 0.82520863, -0.47519908, -0.47432585, ...,  0.        ,
         0.        ,  1.        ],
       ...,
       [ 0.82520863,  0.43135024,  2.00611934, ...,  0.        ,
         0.        ,  1.        ],
       [-1.57221121, -0.47519908, -0.47432585, ...,  1.        ,
         0.        ,  0.        ],
       [ 0.82520863, -0.47519908, -0.47432585, ...,  0.        ,
         1.        ,  0.        ]])

In [31]:
col_transformer_2.fit_transform(X)

array([[ 0.43135024, -0.47432585, -0.50023975, ...,  0.        ,
         1.        ,  3.        ],
       [ 0.43135024, -0.47432585,  0.78894661, ...,  0.        ,
         0.        ,  1.        ],
       [-0.47519908, -0.47432585, -0.48664993, ...,  0.        ,
         1.        ,  3.        ],
       ...,
       [ 0.43135024,  2.00611934, -0.17408416, ...,  0.        ,
         1.        ,  3.        ],
       [-0.47519908, -0.47432585, -0.0422126 , ...,  0.        ,
         0.        ,  1.        ],
       [-0.47519908, -0.47432585, -0.49017322, ...,  1.        ,
         0.        ,  3.        ]])